## Setup

1. In the Azure portal, create a new ***Azure Search*** service in your subscription:
    - Specify a unique URL in the format ***your_search_service_name***.search.windows.net.
    - You can create a new resource group or use an existing one
    - Select the **Basic** pricing tier.
    
    
2. In the Azure portal, create a new ***Storage Account*** service in your subscription:
    - Specify a unique name so that the URL for the account is ***your_storage_account_name***.core.windows.net.
    - Create the storage account in the same location and resource group as your Azure Search service.
    
    
3. After the Storage Account has been created, browse to it in the Azure Portal and view the **Access keys** page, Then copy the **Key** value for *Key1* to the clipboard.Then modify and run the code below to obtain, extract, and upload the documents.

In [1]:
# Azure Storage account settings:
azure_storage_account = 'YOUR_STORAGE_ACCOUNT'
blob_connection_string = "YOUR_BLOB_STORE_CONNECTION_STRING"
blob_container = "YOUR_BLOB_CONTAINER"

# Azure Search constants
azsearch_url = "YOUR_SEARCH_SERVICE.search.windows.net"
azsearch_key = "YOUR_SEARCH_KEY"

In [2]:
# Download and extract document files
!rm -R website_docs
! wget https://aka.ms/kmoh-docs -O web_documents.zip
! unzip -o web_documents.zip

--2019-04-19 00:16:58--  https://aka.ms/kmoh-docs
Resolving webproxy (webproxy)... 10.36.51.1
Connecting to webproxy (webproxy)|10.36.51.1|:3128... connected.
Proxy request sent, awaiting response... 301 Moved Permanently
Location: https://kmohstore.blob.core.windows.net/kmohdata/website_docs.zip [following]
--2019-04-19 00:16:59--  https://kmohstore.blob.core.windows.net/kmohdata/website_docs.zip
Connecting to webproxy (webproxy)|10.36.51.1|:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 9865909 (9.4M) [application/zip]
Saving to: ‘web_documents.zip’

web_documents.zip   100%[===================>]   9.41M  3.00MB/s    in 3.1s    

2019-04-19 00:17:02 (3.00 MB/s) - ‘web_documents.zip’ saved [9865909/9865909]

Archive:  web_documents.zip
   creating: website_docs/collateral/
  inflating: website_docs/collateral/Dubai Brochure.pdf  
  inflating: website_docs/collateral/Las Vegas Brochure.pdf  
  inflating: website_docs/collateral/London Brochure.pdf  
  inflat

In [3]:
# install the Azure Storage SDK
!pip install azure-storage

In [4]:
# Create a new container
from azure.storage.blob import BlockBlobService
try:
    block_blob_service = BlockBlobService(connection_string = blob_connection_string)
    block_blob_service.create_container(blob_container) 
except Exception as ex:
    print(ex)
    
# Copy documents to the new container
import os
local_folder = 'website_docs'
try:
    for root, folders, filenames in os.walk(local_folder):
        for folder in folders:
            for file_name in os.listdir(os.path.join(root,folder)):
                file_path = os.path.join(root,folder, file_name)
                print('Uploading ', file_name)
                block_blob_service.create_blob_from_path(blob_container, file_path, file_path)
    print('done!')
except Exception as ex:
    print(ex)


Uploading  London Brochure.pdf
Uploading  New York Brochure.pdf
Uploading  San Francisco Brochure.pdf
Uploading  Dubai Brochure.pdf
Uploading  Las Vegas Brochure.pdf
Uploading  Margies Travel Company Info.pdf
Uploading  201817.pdf
Uploading  201838.pdf
Uploading  201819.pdf
Uploading  201841.pdf
Uploading  201850.pdf
Uploading  201852.pdf
Uploading  201832.pdf
Uploading  201864.pdf
Uploading  201803.pdf
Uploading  201859.pdf
Uploading  201833.pdf
Uploading  201829.pdf
Uploading  201862.pdf
Uploading  201853.pdf
Uploading  201808.pdf
Uploading  201839.pdf
Uploading  201807.pdf
Uploading  201802.pdf
Uploading  201860.pdf
Uploading  201811.pdf
Uploading  201844.pdf
Uploading  201821.pdf
Uploading  201848.pdf
Uploading  201836.pdf
Uploading  201837.pdf
Uploading  201843.pdf
Uploading  201847.pdf
Uploading  201805.pdf
Uploading  201822.pdf
Uploading  201851.pdf
Uploading  201814.pdf
Uploading  201826.pdf
Uploading  201806.pdf
Uploading  201809.pdf
Uploading  201815.pdf
Uploading  201863.pdf

## Function to wrap HTTP requests

In [5]:
def azsearch_rest(request_type="GET", function_name="servicestats", body=None):
    # Imports and constants
    import http.client, urllib.request, urllib.parse, urllib.error, base64, json, urllib

    # Request headers.
    headers = {
        'Content-Type': 'application/json',
        'api-key': azsearch_key
    }

    # Request parameters
    params = urllib.parse.urlencode({
        'api-version':'2017-11-11-Preview'
    })
    
    try:
        # Execute the REST API call and get the response.
        conn = http.client.HTTPSConnection(azsearch_url)
        request_path = "/{0}?{1}".format(function_name, params)
        conn.request(request_type, request_path, body, headers)
        response = conn.getresponse()
        print(response.status)
        data = response.read().decode("UTF-8")
        result = None
        if len(data) > 0:
            result = json.loads(data)
        return result

    except Exception as ex:
        raise ex

print("Ready to use the REST API")

Ready to use the REST API


## Create a Data Source

In [6]:
import json

# Define the request body
body = {   
    "name" : "website-docs",  
    "description" : "Various documents.",  
    "type" : "azureblob",
    "credentials" :
    { "connectionString" : blob_connection_string
    },  
    "container" : { "name" : blob_container }
} 

try:
    result = azsearch_rest(request_type="POST", function_name="datasources", body=json.dumps(body))
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))
except Exception as ex:
    print(ex)

201
{
  "@odata.context": "https://azsrch-margies.search.windows.net/$metadata#datasources/$entity",
  "@odata.etag": "\"0x8D6C45C6B278563\"",
  "container": {
    "name": "margies-data",
    "query": null
  },
  "credentials": {
    "connectionString": "DefaultEndpointsProtocol=https;AccountName=azstoremargies;AccountKey=vlRSvJgtkncr9+XxwFNno53QZKHU9av0Yu6hKUPj6ZbGPm4mA//Ze8urDczJCDc+11j5SEWhUbHqJR3Ok1m9Wg==;EndpointSuffix=core.windows.net"
  },
  "dataChangeDetectionPolicy": null,
  "dataDeletionDetectionPolicy": null,
  "description": "Various documents.",
  "name": "website-docs",
  "subtype": null,
  "type": "azureblob"
}


In [7]:
import json

try:
    result = azsearch_rest(function_name="datasources")
    if len(result) > 0:
        print(json.dumps(result, sort_keys=True, indent=2))

except Exception as e:
    print('Error:')
    print(e)

200
{
  "@odata.context": "https://azsrch-margies.search.windows.net/$metadata#datasources",
  "value": [
    {
      "@odata.etag": "\"0x8D6C45C6B278563\"",
      "container": {
        "name": "margies-data",
        "query": null
      },
      "credentials": {
        "connectionString": null
      },
      "dataChangeDetectionPolicy": null,
      "dataDeletionDetectionPolicy": null,
      "description": "Various documents.",
      "name": "website-docs",
      "subtype": null,
      "type": "azureblob"
    }
  ]
}


## Create an Index

In [8]:
import json

# Define the request body
body = {
  "fields": [
    {
      # Unique ID for each document
      "name": "id",
      "type": "Edm.String",
      "key": True,
      "searchable": True,
      "filterable": True,
      "facetable": False,
      "sortable": True
    },
    {
     # The file path of the document
      "name": "url",
      "type": "Edm.String",
      "sortable": True,
      "searchable": True,
      "filterable": True,
      "facetable": False
    },
    {
      # The file name of the document
      "name": "file_name",
      "type": "Edm.String",
      "sortable": True,
      "searchable": True,
      "filterable": True,
      "facetable": False
    },
    {
      # The text content in the document
      "name": "content",
      "type": "Edm.String",
      "sortable": False,
      "searchable": True,
      "filterable": True,
      "facetable": False
    },
    {
      # The size of the document
      "name": "size",
      "type": "Edm.Int64",
      "sortable": True,
      "searchable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # The last-modified date of the document
      "name": "last_modified",
      "type": "Edm.DateTimeOffset",
      "sortable": True,
      "searchable": False,
      "filterable": True,
      "facetable": False
    }
  ]
}  

try:
    result = azsearch_rest(request_type="PUT", function_name="indexes/website-docs-index", body=json.dumps(body))
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))

except Exception as e:
    print('Error:')
    print(e)
    

201
{
  "@odata.context": "https://azsrch-margies.search.windows.net/$metadata#indexes/$entity",
  "@odata.etag": "\"0x8D6C45C6C15925A\"",
  "analyzers": [],
  "charFilters": [],
  "corsOptions": null,
  "defaultScoringProfile": null,
  "encryptionKey": null,
  "fields": [
    {
      "analyzer": null,
      "facetable": false,
      "filterable": true,
      "indexAnalyzer": null,
      "key": true,
      "name": "id",
      "retrievable": true,
      "searchAnalyzer": null,
      "searchable": true,
      "sortable": true,
      "synonymMaps": [],
      "type": "Edm.String"
    },
    {
      "analyzer": null,
      "facetable": false,
      "filterable": true,
      "indexAnalyzer": null,
      "key": false,
      "name": "url",
      "retrievable": true,
      "searchAnalyzer": null,
      "searchable": true,
      "sortable": true,
      "synonymMaps": [],
      "type": "Edm.String"
    },
    {
      "analyzer": null,
      "facetable": false,
      "filterable": true,
      "ind

## Create an Indexer

In [9]:
import json

body = {
  "name":"website-docs-indexer", 
  "dataSourceName" : "website-docs",
  "targetIndexName" : "website-docs-index",
  "fieldMappings" : [
      # Map the soure content fields to index fields
        {
          "sourceFieldName" : "metadata_storage_path",
          "targetFieldName" : "id",
          "mappingFunction" : 
            { "name" : "base64Encode" }
        },
        {
          "sourceFieldName" : "metadata_storage_path",
          "targetFieldName" : "url"
        },
        {
          "sourceFieldName" : "metadata_storage_name",
          "targetFieldName" : "file_name"
        },
        {
          "sourceFieldName" : "content",
          "targetFieldName" : "content"
        },
        {
          "sourceFieldName" : "metadata_storage_size",
          "targetFieldName" : "size"
        },
        {
          "sourceFieldName" : "metadata_storage_last_modified",
          "targetFieldName" : "last_modified"
        }
   ],
  "parameters":
  {
    "maxFailedItems":-1,
    "maxFailedItemsPerBatch":-1,
    "configuration": 
    {
        "dataToExtract": "contentAndMetadata"
    }
  }
}  

try:
    result = azsearch_rest(request_type="PUT", function_name="indexers/website-docs-indexer", body=json.dumps(body))
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))

except Exception as e:
    print('Error:')
    print(e)

201
{
  "@odata.context": "https://azsrch-margies.search.windows.net/$metadata#indexers/$entity",
  "@odata.etag": "\"0x8D6C45C6D2BA0D7\"",
  "dataSourceName": "website-docs",
  "description": null,
  "disabled": null,
  "fieldMappings": [
    {
      "mappingFunction": {
        "name": "base64Encode",
        "parameters": null
      },
      "sourceFieldName": "metadata_storage_path",
      "targetFieldName": "id"
    },
    {
      "mappingFunction": null,
      "sourceFieldName": "metadata_storage_path",
      "targetFieldName": "url"
    },
    {
      "mappingFunction": null,
      "sourceFieldName": "metadata_storage_name",
      "targetFieldName": "file_name"
    },
    {
      "mappingFunction": null,
      "sourceFieldName": "content",
      "targetFieldName": "content"
    },
    {
      "mappingFunction": null,
      "sourceFieldName": "metadata_storage_size",
      "targetFieldName": "size"
    },
    {
      "mappingFunction": null,
      "sourceFieldName": "metadata_sto

In [10]:
import time, json

try:
    complete = False
    while (complete == False):
        result = azsearch_rest(request_type="GET", function_name="indexers/website-docs-indexer/status")
        state = result["status"]
        if result['lastResult'] is not None:
            state = result['lastResult']['status']
        print (state)
        if state in ("success", "error"):
            complete = True
        time.sleep(1)

except Exception as e:
    print('Error:')
    print(e)

200
running
200
running
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
success


## Create Wrapper Function for Query Requests

In [11]:
def azsearch_query(index, params):
    # Imports and constants
    import http.client, urllib.request, urllib.parse, urllib.error, base64, json, urllib

    # Request headers.
    headers = {
        'Content-Type': 'application/json',
        'api-key': azsearch_key
    }
    
    try:
        # Execute the REST API call and get the response.
        conn = http.client.HTTPSConnection(azsearch_url)
        request_path = "/indexes/{0}/docs?{1}".format(index, params)
        conn.request("GET", request_path, None, headers)
        response = conn.getresponse()
        data = response.read().decode("UTF-8")
        result = json.loads(data)
        return result

    except Exception as ex:
        raise ex

print("Ready to use the REST API for Queries")

Ready to use the REST API for Queries


## Queries

### Basic Search

In [64]:
import urllib.parse, json

search_terms = input("Search: ")

# Define the search parameters
searchParams = urllib.parse.urlencode({
    # e.g. Search content for "New York"
    'search':'"{0}"'.format(search_terms),
    'searchMode':'All',
    '$count':'true',
    'queryType':'simple',
    '$select':'file_name, url, size, last_modified',
    'api-version':'2017-11-11-Preview'
})

try:
    result = azsearch_query(index="website-docs-index", params=searchParams)
    print('Hits:',result['@odata.count'])
    for value in result["value"]:
        print("\n",value["url"])
        print(value["file_name"], value["size"],"bytes", value["last_modified"])
        if result['@odata.count'] == 1:
            print(value["content"])
        

except Exception as e:
    print('Error:')
    print(e)

Search: London
Hits: 19

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/collateral/London%20Brochure.pdf
London Brochure.pdf 453201 bytes 2019-04-19T00:17:22Z

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201819.pdf
201819.pdf 100657 bytes 2019-04-19T00:17:24Z

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201823.pdf
201823.pdf 107762 bytes 2019-04-19T00:17:29Z

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201855.pdf
201855.pdf 108679 bytes 2019-04-19T00:17:28Z

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201866.pdf
201866.pdf 114948 bytes 2019-04-19T00:17:29Z

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201802.pdf
201802.pdf 112199 bytes 2019-04-19T00:17:26Z

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201811.pdf
201811.pdf 109430 bytes 2019-04-19T00:1

### Sort Results

In [68]:
import urllib.parse, json

search_terms = input("Search: ")

# Define the search parameters
searchParams = urllib.parse.urlencode({
    # e.g. Search content for "New York"
    'search':'"{0}"'.format(search_terms),
    'searchMode':'All',
    '$count':'true',
    'queryType':'simple',
    '$select':'file_name, url, size, last_modified',
    '$orderby': 'last_modified desc',
    'api-version':'2017-11-11-Preview'
})

try:
    result = azsearch_query(index="website-docs-index", params=searchParams)
    print('Hits:',result['@odata.count'])
    for value in result["value"]:
        print("\n",value["url"])
        print(value["file_name"], value["size"],"bytes", value["last_modified"])
        if result['@odata.count'] == 1:
            print(value["content"])
        

except Exception as e:
    print('Error:')
    print(e)

Search: New York
Hits: 20

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201813.pdf
201813.pdf 118903 bytes 2019-04-19T00:17:32Z

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201861.pdf
201861.pdf 117087 bytes 2019-04-19T00:17:31Z

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201828.pdf
201828.pdf 113804 bytes 2019-04-19T00:17:31Z

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201858.pdf
201858.pdf 119046 bytes 2019-04-19T00:17:30Z

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201831.pdf
201831.pdf 120870 bytes 2019-04-19T00:17:30Z

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201825.pdf
201825.pdf 113193 bytes 2019-04-19T00:17:30Z

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201846.pdf
201846.pdf 117367 bytes 2019-04-19T00:17:29Z

 https://azsto

### Wildcards

In [97]:
import urllib.parse, json

search_terms = input("Search (enter * or ? wildcards): ")
# For example, "Earl*s" Court returns Earl's Court, Earls Court, and Earl`s Court

# Define the search parameters
searchParams = urllib.parse.urlencode({
    'search':search_terms,
    'searchMode':'All',
    '$count':'true',
    'queryType':'full',
    '$select':'file_name, content, url, size, last_modified',
    'api-version':'2017-11-11-Preview'
})

try:
    result = azsearch_query(index="website-docs-index", params=searchParams)
    print('Hits:',result['@odata.count'])
    for value in result["value"]:
        print("\n",value["url"])
        print(value["file_name"], value["size"],"bytes", value["last_modified"])
        print(value["content"])

except Exception as e:
    print('Error:')
    print(e)

Search (enter * or ? wildcards): Earl*s Court
Hits: 5

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201830.pdf
201830.pdf 111776 bytes 2019-04-19T00:17:29Z

small room and tiny en-suite but friendly and helpful 

The Kensington Hotel, London, UK 

10/16/2018 

Location is excellent - 4 minute walk from Earls Court tube station. Receptionists were friendly and 

helpful - got us to our room at 11am and looked after our suitcase after checkout while we went to 

the city.Twin room was small but was reasonably clean and had the basics. However ensuite was 

very small 

Reviewer: Abhay Chadalavada 




 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/collateral/London%20Brochure.pdf
London Brochure.pdf 453201 bytes 2019-04-19T00:17:22Z

Margie’s Travel Presents… 

London 
London is the capital and 

most populous city of 

England and the United 

Kingdom. Standing on the 

River Thames in the south 

east of the island of Great 



### Multiple Terms

In [67]:
import urllib.parse, json

search_term1 = input("Search(1): ")
search_term2 = input("Search(2): ")

# Define the search parameters
searchParams = urllib.parse.urlencode({
    # Search for "London" and "Trafalgar Square"
    'search':'"{0}" + "{1}"'.format(search_term1, search_term2),
    'searchMode':'All',
    '$count':'true',
    'queryType':'full',
    '$select':'file_name, url, content, size, last_modified',
    'api-version':'2017-11-11-Preview'
})

try:
    result = azsearch_query(index="website-docs-index", params=searchParams)
    print('Hits:',result['@odata.count'])
    for value in result["value"]:
        print("\n", value["url"])
        print(value["file_name"], value["size"],"bytes", value["last_modified"])
        if result['@odata.count'] == 1:
            print(value["content"])

except Exception as e:
    print('Error:')
    print(e)

Search(1): London
Search(2): Trafalgar Square
Hits: 1

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/collateral/London%20Brochure.pdf
London Brochure.pdf 453201 bytes 2019-04-19T00:17:22Z

Margie’s Travel Presents… 

London 
London is the capital and 

most populous city of 

England and the United 

Kingdom. Standing on the 

River Thames in the south 

east of the island of Great 

Britain, London has been 

a major settlement for two 

millennia. It was founded 

by the Romans, who 

named it Londinium. 

London's ancient core, the 

City of London, largely 

retains its 1.12-square-

mile medieval boundaries. 

Since at least the 19th century, London has also referred to the metropolis around this core, 

historically split between Middlesex, Essex, Surrey, Kent, and Hertfordshire, which today largely 

makes up Greater London, governed by the Mayor of London and the London Assembly. 

 

Mostly popular for: 
Leisure, Outdoors, Historical, Arts 
& Culture 

### Reviews

In [69]:
import urllib.parse, json

search_terms = input("Search: ")

# Define the search parameters
searchParams = urllib.parse.urlencode({
    # Search for "Las vegas"
    'search':'"{0}"'.format(search_terms),
    'searchMode':'All',
    '$count':'true',
    'queryType':'full',
    '$select':'url, file_name, size, last_modified',
    # filter where url includes "reviews"
    '$filter': "search.ismatch('reviews', 'url')",
    'api-version':'2017-11-11-Preview'
})

try:
    result = azsearch_query(index="website-docs-index", params=searchParams)
    print('Hits:',result['@odata.count'])
    for value in result["value"]:
        print("\n",value["url"])
        print(value["file_name"], value["size"],"bytes", value["last_modified"])
        if result['@odata.count'] == 1:
            print(value["content"])

except Exception as e:
    print('Error:')
    print(e)

Search: Las Vegas
Hits: 13

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201853.pdf
201853.pdf 111020 bytes 2019-04-19T00:17:25Z

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201856.pdf
201856.pdf 104470 bytes 2019-04-19T00:17:31Z

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201845.pdf
201845.pdf 116343 bytes 2019-04-19T00:17:31Z

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201844.pdf
201844.pdf 112323 bytes 2019-04-19T00:17:26Z

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201859.pdf
201859.pdf 112112 bytes 2019-04-19T00:17:25Z

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201841.pdf
201841.pdf 109963 bytes 2019-04-19T00:17:24Z

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201857.pdf
201857.pdf 103021 bytes 2019-04-19T00:17:28Z

 https://azst

### Non-reviews

In [70]:
import urllib.parse, json

search_terms = input("Search: ")

# Define the search parameters
searchParams = urllib.parse.urlencode({
    # Search for "Las Vegas" where URL doesn't include "reviews"
    'search':'"{0}" + !url:reviews'.format(search_terms),
    'searchMode':'All',
    '$count':'true',
    'queryType':'full',
    '$select':'file_name, url, size, last_modified',
    'api-version':'2017-11-11-Preview'
})

try:
    result = azsearch_query(index="website-docs-index", params=searchParams)
    print('Hits:',result['@odata.count'])
    for value in result["value"]:
        print("\n",value["url"])
        print(value["file_name"], value["size"],"bytes", value["last_modified"])
        if result['@odata.count'] == 1:
            print(value["content"])

except Exception as e:
    print('Error:')
    print(e)

Search: Las vegas
Hits: 2

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/collateral/Las%20Vegas%20Brochure.pdf
Las Vegas Brochure.pdf 572468 bytes 2019-04-19T00:17:23Z

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/collateral/Margies%20Travel%20Company%20Info.pdf
Margies Travel Company Info.pdf 354991 bytes 2019-04-19T00:17:23Z


### Exclusions

In [71]:
import urllib.parse, json

search_terms = input("Search: ")
exclusion_terms = input("Exclude: ")

# Define the search parameters
searchParams = urllib.parse.urlencode({
    'search':'"{0}" - "{1}"'.format(search_terms, exclusion_terms),
    'searchMode':'All',
    '$count':'true',
    'queryType':'full',
    '$select':'content, url, file_name, size, last_modified',
    'api-version':'2017-11-11-Preview'
})

try:
    result = azsearch_query(index="website-docs-index", params=searchParams)
    print('Hits:',result['@odata.count'])
    for value in result["value"]:
        print("\n",value["url"])
        print(value["file_name"], value["size"],"bytes", value["last_modified"])
        if result['@odata.count'] == 1:
            print(value["content"])

except Exception as e:
    print('Error:')
    print(e)

Search: Dubai
Exclude: The lost City
Hits: 7

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201852.pdf
201852.pdf 106168 bytes 2019-04-19T00:17:24Z

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201864.pdf
201864.pdf 122681 bytes 2019-04-19T00:17:24Z

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/collateral/Dubai%20Brochure.pdf
Dubai Brochure.pdf 391186 bytes 2019-04-19T00:17:23Z

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201862.pdf
201862.pdf 112303 bytes 2019-04-19T00:17:25Z

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201863.pdf
201863.pdf 119467 bytes 2019-04-19T00:17:28Z

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201854.pdf
201854.pdf 106657 bytes 2019-04-19T00:17:28Z

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201851.pdf
201851.pdf 118077 by

### Recent Collateral

In [98]:
import urllib.parse, json

search_terms = input("Search: ")

# Define the search parameters
searchParams = urllib.parse.urlencode({
    'search':'"{0}" + url:collateral'.format(search_terms),
    'searchMode':'All',
    '$count':'true',
    'queryType':'full',
    '$select':'file_name, url, last_modified, size, content',
    '$filter':'last_modified ge 2019-01-01T00:00:00Z',
    'api-version':'2017-11-11-Preview'
})

try:
    result = azsearch_query(index="website-docs-index", params=searchParams)
    print('Hits:',result['@odata.count'])
    for value in result["value"]:
        print("\n",value["url"])
        print(value["file_name"], value["size"],"bytes", value["last_modified"])
        if result['@odata.count'] == 1:
            print(value["content"])

except Exception as e:
    print('Error:')
    print(e)

Search: Las vegas
Hits: 2

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/collateral/Las%20Vegas%20Brochure.pdf
Las Vegas Brochure.pdf 572468 bytes 2019-04-19T00:17:23Z

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/collateral/Margies%20Travel%20Company%20Info.pdf
Margies Travel Company Info.pdf 354991 bytes 2019-04-19T00:17:23Z


### Close Terms

In [72]:
import urllib.parse, json

search_term = input("City: ")
proximity_terms = input("Search Parameters: ")

# Define the search parameters
searchParams = urllib.parse.urlencode({
    # Search for "San Franscisco", and "hotel" and "internet" within 10 words of one another
    'search':'"{0}" + "{1}"~10'.format(search_term, proximity_terms),
    'searchMode':'All',
    '$count':'true',
    'queryType':'full',
    '$select':'file_name, url, content, size, last_modified',
    'api-version':'2017-11-11-Preview'
})

try:
    result = azsearch_query(index="website-docs-index", params=searchParams)
    print('Hits:',result['@odata.count'])
    for value in result["value"]:
        print("\n",value["url"])
        print(value["file_name"], value["size"],"bytes", value["last_modified"])
        if result['@odata.count'] == 1:
            print(value["content"])


except Exception as e:
    print('Error:')
    print(e)

City: San Francisco
Search Parameters: Hotel Internet
Hits: 1

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201810.pdf
201810.pdf 107052 bytes 2019-04-19T00:17:29Z

Excellent value 

The Lombard Hotel, San Francisco, USA 

7/20/2018 

Stayed for three night during 4th of July week-end and was happy with our choice of hotel. The hotel 

is very basic, but free parking, free Internet and an excellent location at a reasonable price makes 

this an excellent choice. 

Reviewer: Gulzar Mehta 





### Fuzzy

In [91]:
import urllib.parse, json

search_terms = input("Reviewer Name: ")

# Define the search parameters
searchParams = urllib.parse.urlencode({
    # Search reviews for "Dubai" but not "The Lost City"
    'search':'{0}~ + url:reviews'.format(search_terms),
    'searchMode':'All',
    '$count':'true',
    'queryType':'full',
    '$select':'file_name, content, url, last_modified, size',
    'api-version':'2017-11-11-Preview'
})

try:
    result = azsearch_query(index="website-docs-index", params=searchParams)
    print('Hits:',result['@odata.count'])
    for value in result["value"]:
        print("\n",value["url"])
        print(value["file_name"], value["size"],"bytes", value["last_modified"])
        if result['@odata.count'] == 1:
            print(value["content"])

except Exception as e:
    print('Error:')
    print(e)

Reviewer Name: Josefine
Hits: 1

 https://azstoremargies.blob.core.windows.net/margies-data/website_docs/reviews/201823.pdf
201823.pdf 107762 bytes 2019-04-19T00:17:29Z

Peaceful 

The City Hotel, London, UK 

10/5/2018 

We stayed in this hotel in late September. Although in busy part of London it was quiet at the 

weekend. Overall the staff were very friendly and the accommodation excellent for our needs 

Reviewer: Josefina Conrad 



